In [2]:
%pip install python-dotenv

In [3]:
%pip install xlsxwriter

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 172.3/172.3 kB 13.4 MB/s eta 0:00:00


In [4]:
import openai
from openai import OpenAI
import os
from dotenv import load_dotenv

In [ ]:
# Load biến môi trường từ file .env
load_dotenv()

# Lấy giá trị biến API_KEY
openai.api_key = os.getenv("API_KEY")

print("API Key:", openai.api_key)

In [6]:
client = OpenAI(api_key=openai.api_key)

In [7]:
import pandas as pd
from google.colab import files

In [8]:
uploaded = files.upload()  # Tải lên file Excel

Saving draft -  Financials 2024.xlsx to draft -  Financials 2024 (1).xlsx


In [9]:
file_name = list(uploaded.keys())[0]
df = pd.read_excel(file_name, header=None)
# df.head(10

mask = df.apply(lambda row: row.astype(str).str.lower().str.contains("code").any(), axis=1)
matches = df[mask]

In [10]:
start_index = matches.index[0]
df_from_code = df.iloc[start_index:].reset_index(drop=True)

# Đặt dòng đầu tiên là header
df_from_code.columns = df_from_code.iloc[0]
df_from_code = df_from_code[1:].reset_index(drop=True)

In [11]:
indicator = pd.read_excel("full_financial_indicators.xlsx")

In [12]:
balance_items = indicator['Balance Sheet'].dropna().tolist()
balance_str = "\n".join(f"- {item}" for item in balance_items)

In [13]:
income_items = indicator['Income Statement'].dropna().tolist()
income_str = "\n".join(f"- {item}" for item in income_items)

In [14]:
cf_items = indicator['Cash Flow Statement'].dropna().tolist()
cf_str = "\n".join(f"- {item}" for item in cf_items)

In [ ]:
## 4. Tạo Prompt từ bảng dữ liệu
def generate_prompt_from_df(df):
    table_str = df.to_string(index=False)
    prompt = f"""
    You are a financial analyst. Below is a table of daily financial data:

    {table_str}

    Please:
    1. Generate a **professional financial summary report** based on this data according to Vietnamese accounting standard Circular 200/2014/TT-BTC.
    2. Include key sections in the report: Revenue, Costs, Profit, and meaningful comments.
    3. Additionally, output a **clean and concise Excel-style table** (with headers) containing only the **main financial indicators**, such as:
      - Total Revenue
      - Total Cost of Goods Sold
      - Total Operating Expenses
      - Total Net Profit

    The output should contain two parts:
    - A human-readable report summary
    - A clearly formatted financial summary table, ready to export to Excel

    Use professional English.
    """
    return prompt

In [20]:
def generate_prompt_from_df(df):
    table_str = df.to_string(index=False)
    prompt = f"""
    You are a financial analyst. Below is a table of daily financial data, which contains two key columns representing financial figures for two years.
    The columns may have names such as "2023" and "2024", or "Last Year" and "Current Year", or similar variants.

    {table_str}

    Below is a list of financial indicators that belong to the Balance Sheet section:

    Balance Sheet Indicators:
    {balance_str}

    Income Statement Indicators:
    {income_str}

    Cash Flow Statement Indicators:
    {cf_str}

    Please:

    1. Automatically detect and use the two columns that represent the two years (previous year and current year) to extract numeric data for all calculations.

    2. Calculate all main financial indicators for both years based on these two columns.

    3. Organize the financial indicators into three separate, clean Excel-style tables:
    Each table must begin with a heading in the following format:
     #### Balance Sheet Table
     #### Income Statement Table
     #### Cash Flow Statement Table

    4. Each table must include two columns with numeric values: one for the current year and one for the previous year, enabling year-over-year comparison.

    5. Generate a professional, human-readable financial summary report in English that highlights:
      - Key revenue figures
      - Cost and expense analysis
      - Profitability overview
      - Cash flow performance
      - Meaningful comments on significant changes between the two years, based strictly on the numeric data from the two year columns

    6. Avoid including qualitative or vague comments inside the tables—only present numeric financial figures.

    7. Output two parts:
      - A concise, professional financial summary report in English
      - Three clearly formatted Excel-style tables with numeric data for both years side-by-side, ready for export

    Use professional English.
    """
    return prompt

In [21]:
prompt = generate_prompt_from_df(df_from_code)
print(prompt)


    You are a financial analyst. Below is a table of daily financial data, which contains two key columns representing financial figures for two years.
    The columns may have names such as "2023" and "2024", or "Last Year" and "Current Year", or similar variants.

                  Code                       Description                 Grouping                        Main Group Opening Balance       Debit      Credit  Difference Closing Balance 2024  NaN Closing Balance 2023
            Assets                               NaN                       NaN                              NaN             NaN         NaN         NaN         NaN                  NaN  NaN                  NaN
             11046 CASHIER FLOAT - GRAND BEIRUT EXPO             Cash In hand                             Cash             1000           0           0           0                 1000  NaN                 1000
             12302            INTER CO ACCTS  REC. -     Inter Company balance                 

In [22]:
## 5. Gọi GPT để sinh báo cáo
response = client.chat.completions.create(
    model="gpt-4o",
    messages=[
        {"role": "system", "content": "You are a financial analyst"},
        {"role": "user", "content": prompt}
    ]
)

report_text = response.choices[0].message.content
print(report_text)

### Financial Summary Report

#### Key Revenue Figures
In the current year, the total income generated was significantly lower as compared to the previous year. The revenue from food and beverage sales saw marked declines, with food revenue at -$1183287.83 in the current year compared to -$1606930.44 in the previous year, and beverage revenues continuing to show negative figures for both alcoholic and non-alcoholic categories.

#### Cost and Expense Analysis
The expenses in the current year have increased across several areas, leading to a total expense of $1809703.73, compared to $2023888.3 in the previous year. Notable increases were observed in salaries and wages, particularly the basic salary and overtime expenses. Costs associated with utilities such as electricity and gas also saw an increment, signifying an increase in operational expenses.

#### Profitability Overview
The significant decline in revenue resulted in decreased profitability for the current year. Despite efforts to

In [ ]:
# 7. Trích bảng từ output GPT (đơn giản)
import re
lines = report_text.splitlines()
table_lines = [l for l in lines if "|" in l and "---" not in l]

# Bỏ dấu | đầu cuối rồi mới tách theo dấu |
rows = [re.split(r"\s*\|\s*", l.strip().strip('|')) for l in table_lines]

if len(rows) > 1:
    df_summary = pd.DataFrame(rows[1:], columns=rows[0])
    df_summary.to_excel("financial_summary.xlsx", index=False)
    files.download("financial_summary.xlsx")
else:
    print("⚠️ No summary table found.")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [23]:
import re
import pandas as pd

# Giả sử report_text là nội dung text từ GPT
lines = report_text.splitlines()

tables = {}
current_table_name = None
table_lines = []

def save_table(name, lines):
    rows = [re.split(r"\s*\|\s*", l.strip().strip('|')) for l in lines]
    if len(rows) > 1:
        df = pd.DataFrame(rows[1:], columns=rows[0])
        tables[name] = df

for line in lines:
    # Nhận tiêu đề dạng: #### Balance Sheet Table
    match = re.match(r"^####\s+(.*?)\s+Table", line.strip())
    if match:
        # Nếu có bảng trước đó, lưu lại
        if current_table_name and table_lines:
            save_table(current_table_name, table_lines)
            table_lines = []

        # Cập nhật tên bảng
        current_table_name = match.group(1)

    elif "|" in line and "---" not in line:
        table_lines.append(line)

# Lưu bảng cuối cùng nếu có
if current_table_name and table_lines:
    save_table(current_table_name, table_lines)

# Ghi ra file Excel với nhiều sheet
if tables:
    with pd.ExcelWriter("financial_statements_from_gpt.xlsx", engine="xlsxwriter") as writer:
        for sheet_name, df in tables.items():
            df.to_excel(writer, sheet_name=sheet_name, index=False)
    files.download("financial_statements_from_gpt.xlsx")
    print("✅ Saved to financial_statements_from_gpt.xlsx")
else:
    print("⚠️ No tables found in report_text.")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

✅ Saved to financial_statements_from_gpt.xlsx


In [ ]:
prompt = f'How do you calculate Total Cost of Goods Sold from {df}'
response = client.chat.completions.create(
    model="gpt-4o",
    messages=[
        {"role": "system", "content": "You are a financial analyst"},
        {"role": "user", "content": prompt}
    ]
)

report_text = response.choices[0].message.content
print(report_text)

To calculate the Total Cost of Goods Sold (COGS) from the provided data, we need to identify and sum up all the costs directly associated with the production and delivery of goods sold by a company. These costs typically include raw materials, components, labor, and manufacturing overhead incurred in making the product.

In the dataset you provided, you'll need to determine which transactions represent these types of costs. From the sample data you provided, it seems like transaction IDs and their descriptions vary but generally don't fully describe typical COGS transactions directly. However, we can make some assumptions:

1. **Payment to suppliers**: This typically forms part of COGS if it is related to purchasing materials or components needed for production. The transaction on "2025-01-02" with the description "Payment to supplier" for the amount "-3000" likely relates to COGS because it involves a supplier payment.

2. Other transactions: Cash transactions and accounts receivable 

In [ ]:
# @title Performance metrics


1. Exact Match Accuracy (EMA) <br>
Compares the entire extracted table from GPT with the ground truth. If every cell matches exactly, the prediction is considered 100% correct.
```python

        `accuracy = (df_output.equals(df_ground_truth))`

2. Cell-level Accuracy / Match Rate <br>
Compares each individual cell in the predicted table against the corresponding cell in the reference table.
```python

          `total_cells = df_output.size`
          `matched_cells = (df_output.values == df_ground_truth.values).sum()`
          `cell_accuracy = matched_cells / total_cells`

3. Levenshtein Similarity / Fuzzy Matching
Uses fuzzy string matching (via fuzzywuzzy or rapidfuzz) to allow small variations in values, such as punctuation or formatting differences.
```python

      `from rapidfuzz import fuzz`
      `fuzz.ratio("6,000", "6000")  # ~95% similarity`


In [ ]:
# Cài đặt thư viện cần thiết (nếu chưa có)
!pip install datasets --quiet

from datasets import load_dataset
import pandas as pd

In [ ]:
df = pd.read_csv("hf://datasets/KadamParth/NCERT_Accounting_11th/Accountancy_11th_Cleaned.csv")